In [1]:
import pandas as pd
import numpy as np
import psycopg2 as pg

In [2]:
#load data as dataframe to chcek struct and column names
cancer = pd.read_csv('/Users/jsong/Documents/cancer_classification/kag_risk_factors_cervical_cancer.csv')

In [3]:
cancer.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0,0,0,0,0,0,0,0


Tried to setup sql database, but missing values with '?' cause error which is preventing copy data to sql database. So need to clean up the data first using pandas, and transfer to sql

In [4]:
#clear column names
cancer.columns = cancer.columns.str.replace(' ', '_')
cancer.columns = cancer.columns.str.replace(':', '_')
cancer.columns = cancer.columns.str.replace('(', '')
cancer.columns = cancer.columns.str.replace(')', '')

In [5]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 36 columns):
Age                                   858 non-null int64
Number_of_sexual_partners             858 non-null object
First_sexual_intercourse              858 non-null object
Num_of_pregnancies                    858 non-null object
Smokes                                858 non-null object
Smokes_years                          858 non-null object
Smokes_packs/year                     858 non-null object
Hormonal_Contraceptives               858 non-null object
Hormonal_Contraceptives_years         858 non-null object
IUD                                   858 non-null object
IUD_years                             858 non-null object
STDs                                  858 non-null object
STDs_number                           858 non-null object
STDs_condylomatosis                   858 non-null object
STDs_cervical_condylomatosis          858 non-null object
STDs_vaginal_condylomatosi

In [7]:
#change '?' values to Nan
cancer.replace(['?'], np.nan)

#change values to numeric
for col in  cancer.columns[1:]:
    cancer[col] = pd.to_numeric(cancer[col], errors='coerce')

In [8]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 36 columns):
Age                                   858 non-null int64
Number_of_sexual_partners             832 non-null float64
First_sexual_intercourse              851 non-null float64
Num_of_pregnancies                    802 non-null float64
Smokes                                845 non-null float64
Smokes_years                          845 non-null float64
Smokes_packs/year                     845 non-null float64
Hormonal_Contraceptives               750 non-null float64
Hormonal_Contraceptives_years         750 non-null float64
IUD                                   741 non-null float64
IUD_years                             741 non-null float64
STDs                                  753 non-null float64
STDs_number                           753 non-null float64
STDs_condylomatosis                   753 non-null float64
STDs_cervical_condylomatosis          753 non-null float64
STDs_vaginal

In [9]:
#fill-out missing values
#for below 3 columns, mean values of each columns are used to fill the missing values
cancer['Number_of_sexual_partners'].fillna((cancer['Number_of_sexual_partners'].mean()), inplace=True)
cancer['First_sexual_intercourse'].fillna((cancer['First_sexual_intercourse'].mean()), inplace=True)
cancer['Num_of_pregnancies'].fillna((cancer['Num_of_pregnancies'].mean()), inplace=True)

In [10]:
#fill with 0 for missing values of other columns 
for col in  cancer.columns[4:]:
    cancer[col].fillna(0, inplace=True)

In [11]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 36 columns):
Age                                   858 non-null int64
Number_of_sexual_partners             858 non-null float64
First_sexual_intercourse              858 non-null float64
Num_of_pregnancies                    858 non-null float64
Smokes                                858 non-null float64
Smokes_years                          858 non-null float64
Smokes_packs/year                     858 non-null float64
Hormonal_Contraceptives               858 non-null float64
Hormonal_Contraceptives_years         858 non-null float64
IUD                                   858 non-null float64
IUD_years                             858 non-null float64
STDs                                  858 non-null float64
STDs_number                           858 non-null float64
STDs_condylomatosis                   858 non-null float64
STDs_cervical_condylomatosis          858 non-null float64
STDs_vaginal

In [12]:
#save cleaned dataframe to csv file, so that copy them to sql database
export_csv = cancer.to_csv(r'/Users/jsong/Documents/cancer_classification/cancer.csv', index = None, header=True)

# transfer data to sql on AWS

CREATE DATABASE cancer;

\connect cancer;

CREATE TABLE cer_cancer(  
age INT,  
num_of_partner FLOAT,  
first_intercourse FLOAT,  
num_of_preg FLOAT,  
smokes FLOAT,  
smokes_years FLOAT,  
smokes_packs FLOAT,  
hormonal_contraceptives FLOAT,  
hormonal_contraceptives_years FLOAT,  
iud FLOAT,  
iud_years FLOAT,  
stds FLOAT,  
stds_num FLOAT,  
condylomatosis FLOAT,  
cervical_condylomatosis FLOAT,  
vaginal_condylomatosis FLOAT,  
vulvo_perineal_condylomatosis FLOAT,  
Syphilis FLOAT,  
pelvic_inflammation FLOAT,  
genital_herpes FLOAT,  
molluscum_contagiosum FLOAT,  
aids FLOAT,  
hiv FLOAT,  
hepatitis_b FLOAT,  
hpv FLOAT,  
num_of_stds FLOAT,  
time_since_std FLOAT,  
time_since_last_std FLOAT,  
dx_cancer INT,  
dx_cin INT,  
dx_hpv INT,  
dx INT,  
hinselmann INT,  
schiller INT,  
cytology INT,  
biopsy INT  
);  
  
    
\copy cer_cancer FROM '/home/ubuntu/cancer_classification/cancer.csv' DELIMITER ',' CSV HEADER;


In [16]:
connection_args = {
    'host': '18.144.83.179',  # You will need to put the PUBLIC IP address of your AWS instance here
    'user': 'ubuntu',    # your username on AWS is 'ubuntu'
    'dbname': 'cancer',    # DB that we are connecting to
    'port': 5432             # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)